# 1. Setup and Get Data

### 1.1 Install Dependencies and Setup

In [ ]:
# !pip install labelme tensorflow tensorflow-gpu opencv-python matplotlib albumentations

### 1.2 Collect Images Using OpenCV

In [ ]:
import os
import time
import uuid
import cv2

In [ ]:
IMAGES_PATH = os.path.join('data','images')
number_images = 30

In [ ]:
cap = cv2.VideoCapture(0)
for imgnum in range(number_images):
    print('Collecting image {}'.format(imgnum))
    ret, frame = cap.read()
    imgname = os.path.join(IMAGES_PATH,f'{str(uuid.uuid1())}.jpg')
    cv2.imwrite(imgname, frame)
    cv2.imshow('frame', frame)
    time.sleep(0.5)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

### 1.3 Annotate Images with LabelMe

In [ ]:
!labelme

# 2. Review Dataset and Build Image Loading Function

### 2.1 Import TF and Deps

In [ ]:
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path

### 2.2 Limit GPU Memory Growth

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.config.list_physical_devices('GPU')

### 2.3 Load Image into TF Data Pipeline

In [ ]:
data_wd = Path('/data') / 'raspi_face_detection'
img_path = data_wd / 'images'
labels_path = data_wd / 'labels'

In [ ]:
images = tf.data.Dataset.list_files(f'{img_path}/*.jpg', shuffle=False)

In [ ]:
images.as_numpy_iterator().next()

In [ ]:
def load_image(x): 
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

In [ ]:
images = images.map(load_image)

In [ ]:
images.as_numpy_iterator().next()

In [ ]:
type(images)

### 2.4 View Raw Images with Matplotlib

In [ ]:
image_generator = images.batch(4).as_numpy_iterator()

In [ ]:
plot_images = image_generator.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, image in enumerate(plot_images):
    ax[idx].imshow(image) 
plt.show()

# 3. Partition Unaugmented Data

### 3.1 MANUALLY SPLT DATA INTO TRAIN TEST AND VAL

In [ ]:
90*.7 # 63 to train

In [ ]:
90*.15 # 14 and 13 to test and val

### 3.1.1 Programmatically Split Data into Train/Test/Val

In [ ]:
from sklearn.model_selection import train_test_split
import json
import pandas as pd
import shutil

In [ ]:
labels = list()
for label_fn in labels_path.glob('*'):
    with open(label_fn, 'r') as f:
        label_json = json.load(f)
    labels.append(dict(
        image_path=Path(label_json['imagePath']).name,
        labels=', '.join(sorted([x['label'] for x in label_json['shapes']]))
    ))

In [ ]:
df_labels = pd.DataFrame(labels)
df_labels.labels.value_counts()

In [ ]:
train, test = train_test_split(df_labels, test_size=0.3, random_state=42, stratify=df_labels.labels)
test, val = train_test_split(test, test_size=0.5, random_state=42, stratify=test.labels)

In [ ]:
print('Training Set')
print(train.labels.value_counts().to_string(header=False))
print('\nTest Set')
print(test.labels.value_counts().to_string(header=False))
print('\nValidation Set')
print(val.labels.value_counts().to_string(header=False))

In [ ]:
model_sets_files = dict(
    train=train.image_path.tolist(),
    test=test.image_path.tolist(),
    val=val.image_path.tolist(),
)
print('Model Set Percentages Check')
print({k: len(v) for k, v in model_sets_files.items()})
print({k: len(v) / len(df_labels) for k, v in model_sets_files.items()})

### 3.2 Moving Images and Labels

In [ ]:
model_sets_path = data_wd / 'model_sets'

In [ ]:
list(model_sets_path.glob('*'))

In [ ]:
folder = 'train'
model_set_filepath = model_sets_path / folder
model_set_filepath_images = model_set_filepath / 'images'
model_set_filepath_labels = model_set_filepath / 'labels'
for p in [model_set_filepath, model_set_filepath_images, model_set_filepath_labels]:
    p.mkdir(exist_ok=True)

In [ ]:
# for folder in ['train','test','val']:
#     model_set_filepath = model_sets_path / folder
#     model_set_filepath_images = model_set_filepath / 'images'
#     model_set_filepath_labels = model_set_filepath / 'labels'
#     for p in [model_set_filepath, model_set_filepath_images, model_set_filepath_labels]:
#         p.mkdir(exist_ok=True)

#     for img_filename in model_sets_files[folder]:
#         src_img_filepath, dst_img_filepath = [p / img_filename for p in [img_path, model_set_filepath_images]]
#         shutil.copyfile(src_img_filepath, dst_img_filepath)
        
#         label_filename = img_filename.split('.')[0]+'.json'
#         src_label_filepath, dst_label_filepath = [p / label_filename for p in [labels_path, model_set_filepath_labels]]
#         shutil.copyfile(src_label_filepath, dst_label_filepath)

# 4. Apply Image Augmentation on Images and Labels using Albumentations

In [ ]:
import albumentations as alb
import cv2

### 4.1 Setup Albumentations Transform Pipeline

In [ ]:
augmentor = alb.Compose([alb.RandomCrop(width=450, height=450), 
                         alb.HorizontalFlip(p=0.5), 
                         alb.RandomBrightnessContrast(p=0.2),
                         alb.RandomGamma(p=0.2), 
                         alb.RGBShift(p=0.2), 
                         alb.VerticalFlip(p=0.5)], 
                         bbox_params=alb.BboxParams(format='albumentations', 
                                                    label_fields=['class_labels']))

### 4.2 Load a Test Image and Annotation with OpenCV and JSON

In [ ]:
fns = list(model_set_filepath_images.glob('*'))
# fn = fns[1].stem
fn = '49e246c6-0968-11ef-b027-dca632a68397'
fn

In [ ]:
model_sets_path

In [ ]:
img_fn   = model_sets_path / 'train' / 'images' / f'{fn}.jpg'
label_fn = model_sets_path / 'train' / 'labels' / f'{fn}.json'

In [ ]:
img = cv2.imread(str(img_fn))
print(('height', 'width', 'layers'))
img.shape

In [ ]:
with open(label_fn, 'r') as f:
    label = json.load(f)

In [ ]:
label.keys()

### 4.3 Extract Coordinates and Rescale to Match Image Resolution

In [ ]:
# coords = [0,0,0,0]
# shapes = label['shapes']
# points = shapes[0]['points']
# coords[0] = points[0][0]
# coords[1] = points[0][1]
# coords[2] = points[1][0]
# coords[3] = points[1][1]

# label = shapes[0]['label']

# print(['p1-width', 'p1-height', 'p2-width', 'p2-height'])
# print(coords)

# print("\n\np1 = top-left point")
# print("p2 = lower-right point")

In [ ]:
# coords = list(np.divide(coords, [640,480,640,480]))

In [ ]:
shapes, coords, labels = label['shapes'], list(), list()
for shape in shapes:
    c = [0,0,0.00001,0.00001]
    c[0] = shape['points'][0][0]
    c[1] = shape['points'][0][1]
    c[2] = shape['points'][1][0]
    c[3] = shape['points'][1][1]
    c = list(np.divide(c, [640,480,640,480]))

    labels.append(shape['label'])
    coords.append(c)
coords, labels

### 4.4 Apply Augmentations and View Results

In [ ]:
augmented = augmentor(image=img, bboxes=coords, class_labels=labels)

In [ ]:
for bbox in augmented['bboxes']:
    cv2.rectangle(augmented['image'], 
                  tuple(np.multiply(bbox[:2], [450,450]).astype(int)),
                  tuple(np.multiply(bbox[2:], [450,450]).astype(int)), 
                        (255,0,0), 2)

plt.imshow(augmented['image'])

### 4.5 Find Issues where x_max is less than or equal to x_min

In [ ]:
errors = list()

fns = list(img_path.glob('*'))
for fn in fns:
    stem = fn.stem
    img_fn   = img_path / f'{stem}.jpg'
    label_fn = labels_path / f'{stem}.json'
    
    
    img = cv2.imread(str(img_fn))
    with open(label_fn, 'r') as f:
        label = json.load(f)
    
    shapes = label['shapes']
    for shape in shapes:
        coords = [0,0,0,0]
        points = shape['points']
        coords[0] = points[0][0]
        coords[1] = points[0][1]
        coords[2] = points[1][0]
        coords[3] = points[1][1]
        coords = list(np.divide(coords, [640,480,640,480]))
    
        try:
            augmented = augmentor(image=img, bboxes=[coords], class_labels=['face'])
        except ValueError:
            errors.append([stem, shape['label']])

print(len(errors))

In [ ]:
print(pd.DataFrame(errors).to_csv())

# 5. Build and Run Augmentation Pipeline

In [ ]:
aug_sets_path = data_wd / 'aug_data'

### 5.1 Run Augmentation Pipeline

In [ ]:
# class_nums = dict(brendan=1, kara=2)

# for partition in ['train','test','val']: 
#     model_set_filepath = model_sets_path / partition
#     model_set_filepath_images = model_set_filepath / 'images'
#     model_set_filepath_labels = model_set_filepath / 'labels'

#     aug_set_filepath = aug_sets_path / partition
#     aug_set_filepath_images = aug_set_filepath / 'images'
#     aug_set_filepath_labels = aug_set_filepath / 'labels'
    
#     for image in list(model_set_filepath_images.glob('*')):
#         img_path = model_set_filepath_images / image
#         img = cv2.imread(str(img_path))

#         label_path = model_set_filepath_labels / f'{img_path.stem}.json'
#         if label_path.is_file():
#             with open(label_path, 'r') as f:
#                 label = json.load(f)

#         shapes, coords, labels = label['shapes'], list(), list()
#         shapes.sort(key=lambda x: x['label'])
#         for shape in shapes:
#             c = [0,0,0.00001,0.00001]
#             c[0] = shape['points'][0][0]
#             c[1] = shape['points'][0][1]
#             c[2] = shape['points'][1][0]
#             c[3] = shape['points'][1][1]
#             c = list(np.divide(c, [640,480,640,480]))
        
#             labels.append(shape['label'])
#             coords.append(c)


#         for x in range(60):
#             augmented = augmentor(image=img, bboxes=coords, class_labels=labels)
#             aug_image, aug_labels, aug_bboxes = augmented['image'], augmented['class_labels'], augmented['bboxes']
            
#             cv2.imwrite(str(aug_set_filepath_images / f'{img_path.stem}_{x}.jpg'), aug_image)

#             annotation = {}
#             annotation['image'] = str(image)

#             if not label_path.is_file() or len(aug_labels) == 0: 
#                 annotation['bboxes'] = [[0,0,0,0], [0,0,0,0]]
#                 annotation['classes'] = [0] 
#             else:
#                 if len(aug_labels) == 1:
#                     if aug_labels[0] == 'kara':
#                         aug_bboxes.insert(0, (0, 0, 0, 0))
#                     else:
#                         aug_bboxes.append((0, 0, 0, 0))
                        
#                 assert len(aug_bboxes) == 2
#                 assert len(aug_bboxes[0]) == 4
#                 assert len(aug_bboxes[1]) == 4
                
#                 annotation['bboxes'] = aug_bboxes
#                 annotation['classes'] = list(map(lambda x: class_nums[x], aug_labels))
#                 # Add new code here
                    


#             with open(str(aug_set_filepath_labels / f'{img_path.stem}_{x}.json'), 'w') as f:
#                 json.dump(annotation, f)

In [ ]:
for partition in ['train','test','val']: 
    aug_set_filepath = aug_sets_path / partition
    aug_set_filepath_images = aug_set_filepath / 'images'
    aug_set_filepath_labels = aug_set_filepath / 'labels'

    cnt_images, cnt_labels = len(list(aug_set_filepath_images.glob('*'))), len(list(aug_set_filepath_labels.glob('*')))

    print(f'{partition} augmentation set' + '\n' + '-'*10)
    print(f'{cnt_images} Images & {cnt_labels} Labels\n\n')

### 5.2 Load Augmented Images to Tensorflow Dataset

In [ ]:
train_images = tf.data.Dataset.list_files(str(aug_sets_path / 'train' / 'images'/ '*.jpg'), shuffle=False)
train_images = train_images.map(load_image)
train_images = train_images.map(lambda x: tf.image.resize(x, (120,120)))
train_images = train_images.map(lambda x: x/255)

In [ ]:
test_images = tf.data.Dataset.list_files(str(aug_sets_path / 'test' / 'images'/ '*.jpg'), shuffle=False)
test_images = test_images.map(load_image)
test_images = test_images.map(lambda x: tf.image.resize(x, (120,120)))
test_images = test_images.map(lambda x: x/255)

In [ ]:
val_images = tf.data.Dataset.list_files(str(aug_sets_path / 'val' / 'images'/ '*.jpg'), shuffle=False)
val_images = val_images.map(load_image)
val_images = val_images.map(lambda x: tf.image.resize(x, (120,120)))
val_images = val_images.map(lambda x: x/255)

In [ ]:
train_images.as_numpy_iterator().next()

# 6. Prepare Labels

### 6.1 Build Label Loading Function

In [ ]:
import itertools

In [ ]:
# fn = '49e246c6-0968-11ef-b027-dca632a68397_5.json' # both
# fn = '3a38b642-0968-11ef-b027-dca632a68397_4.json' # kara
# fn = '4a2f0b44-0965-11ef-a3f3-dca632a68397_37.json' # brendan
fn = '369aab30-0968-11ef-b027-dca632a68397_0.json' # error
label_path = aug_sets_path / 'train' / 'labels' / fn
with open(label_path, 'r', encoding = "utf-8") as f:
    label = json.load(f)

label['classes'], label['bboxes']

In [ ]:
def load_labels(label_path):
    with open(label_path.numpy(), 'r', encoding = "utf-8") as f:
        label = json.load(f)

    return [int(x in label['classes']) for x in range(1, 3)], list(itertools.chain(*label['bboxes']))

# classes, bboxes = load_labels(label_path)

### 6.2 Load Labels to Tensorflow Dataset

In [ ]:
train_labels = tf.data.Dataset.list_files(str(aug_sets_path / 'train' / 'labels'/ '*.json'), shuffle=False)
train_labels = train_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [ ]:
test_labels = tf.data.Dataset.list_files(str(aug_sets_path / 'test' / 'labels'/ '*.json'), shuffle=False)
test_labels = test_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [ ]:
val_labels = tf.data.Dataset.list_files(str(aug_sets_path / 'val' / 'labels'/ '*.json'), shuffle=False)
val_labels = val_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [ ]:
train_labels.as_numpy_iterator().next()

# 7. Combine Label and Image Samples

### 7.1 Check Partition Lengths

In [ ]:
len(train_images), len(train_labels), len(test_images), len(test_labels), len(val_images), len(val_labels)

### 7.2 Create Final Datasets (Images/Labels)

In [ ]:
train = tf.data.Dataset.zip((train_images, train_labels))
train = train.shuffle(5000)
train = train.batch(16)
train = train.prefetch(4)

In [ ]:
test = tf.data.Dataset.zip((test_images, test_labels))
test = test.shuffle(1300)
test = test.batch(16)
test = test.prefetch(4)

In [ ]:
val = tf.data.Dataset.zip((val_images, val_labels))
val = val.shuffle(1000)
val = val.batch(16)
val = val.prefetch(4)

### 7.3 View Images and Annotations

In [ ]:
data_samples = train.as_numpy_iterator()

In [ ]:
res = data_samples.next()
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = res[0][idx]
    sample_coords = res[1][1][idx]
    sample_labels = ', '.join(res[1][0][idx].astype(str).tolist())

    cv2.rectangle(sample_image, 
                  tuple(np.multiply(sample_coords[0:2], [120,120]).astype(int)),
                  tuple(np.multiply(sample_coords[2:4], [120,120]).astype(int)), (255,0,0), 2)
    cv2.rectangle(sample_image, 
                  tuple(np.multiply(sample_coords[4:6], [120,120]).astype(int)),
                  tuple(np.multiply(sample_coords[6:8], [120,120]).astype(int)), (0,255,0), 2)

    ax[idx].imshow(sample_image)
    ax[idx].set_title(sample_labels)

# 8. Build Deep Learning using the Functional API

### 8.1 Import Layers and Base Network

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16

### 8.2 Download VGG16

In [ ]:
vgg = VGG16(include_top=False)

In [ ]:
vgg.summary()

### 8.3 Build instance of Network

In [ ]:
def build_model(): 
    input_layer = Input(shape=(120,120,3))
    
    vgg = VGG16(include_top=False)(input_layer)

    # Classification Model  
    f1 = GlobalMaxPooling2D()(vgg)
    class1 = Dense(2048, activation='relu')(f1)
    class2 = Dense(2, activation='sigmoid')(class1)
    
    # Bounding box model
    f2 = GlobalMaxPooling2D()(vgg)
    regress1 = Dense(2048, activation='relu')(f2)
    regress2 = Dense(8, activation='sigmoid')(regress1)
    
    facetracker = Model(inputs=input_layer, outputs=[class2, regress2])
    return facetracker

### 8.4 Test out Neural Network

In [ ]:
facetracker = build_model()

In [ ]:
facetracker.summary()

In [ ]:
X, y = train.as_numpy_iterator().next()

In [ ]:
X.shape

In [ ]:
classes, coords = facetracker.predict(X)

In [ ]:
classes, coords

# 9. Define Losses and Optimizers

### 9.1 Define Optimizer and LR

In [ ]:
batches_per_epoch = len(train)
lr_decay = (1./0.75 -1)/batches_per_epoch
lr_decay

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.00005, decay=lr_decay)

### 9.2 Create Localization Loss and Classification Loss

In [ ]:
def localization_loss(y_true, yhat):
    # differences between coordinates
    delta_coord = tf.reduce_sum(tf.square(y_true[:,:2] - yhat[:,:2])) + tf.reduce_sum(tf.square(y_true[:,4:6] - yhat[:,4:6]))

    # differences between size
    delta_size = 0
    for i in range(2):
        h_true = y_true[:,3+4*i] - y_true[:,1+4*i] 
        w_true = y_true[:,2+4*i] - y_true[:,0+4*i] 
    
        h_pred = yhat[:,3+4*i] - yhat[:,1+4*i] 
        w_pred = yhat[:,2+4*i] - yhat[:,0+4*i] 
        
        delta_size += tf.reduce_sum(tf.square(w_true - w_pred) + tf.square(h_true-h_pred))
    
    return delta_coord + delta_size

In [ ]:
y_true = [0]*8
y_true

In [ ]:
classloss = tf.keras.losses.BinaryCrossentropy()
regressloss = localization_loss

### 9.3 Test out Loss Metrics

In [ ]:
localization_loss(y[1], coords)

In [ ]:
classloss(y[0], classes)

In [ ]:
regressloss(y[1], coords)

# 10. Train Neural Network

### 10.1 Create Custom Model Class

In [ ]:
class FaceTracker(Model): 
    def __init__(self, eyetracker,  **kwargs): 
        super().__init__(**kwargs)
        self.model = eyetracker

    def compile(self, opt, classloss, localizationloss, **kwargs):
        super().compile(**kwargs)
        self.closs = classloss
        self.lloss = localizationloss
        self.opt = opt
    
    def train_step(self, batch, **kwargs): 
        
        X, y = batch
        
        with tf.GradientTape() as tape: 
            classes, coords = self.model(X, training=True)
            
            batch_classloss = self.closs(y[0], classes)
            batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)
            
            total_loss = batch_localizationloss+0.5*batch_classloss
            
            grad = tape.gradient(total_loss, self.model.trainable_variables)
        
        opt.apply_gradients(zip(grad, self.model.trainable_variables))
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
    
    def test_step(self, batch, **kwargs): 
        X, y = batch
        
        classes, coords = self.model(X, training=False)
        
        batch_classloss = self.closs(y[0], classes)
        batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)
        total_loss = batch_localizationloss+0.5*batch_classloss
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
        
    def call(self, X, **kwargs): 
        return self.model(X, **kwargs)

In [ ]:
model = FaceTracker(facetracker)

In [ ]:
model.compile(opt, classloss, regressloss)

### 10.2 Train

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

### 10.3 Plot Performance

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(20,5))

ax[0].plot(hist.history['total_loss'], color='teal', label='loss')
ax[0].plot(hist.history['val_total_loss'], color='orange', label='val loss')
ax[0].title.set_text('Loss')
ax[0].legend()

ax[1].plot(hist.history['class_loss'], color='teal', label='class loss')
ax[1].plot(hist.history['val_class_loss'], color='orange', label='val class loss')
ax[1].title.set_text('Classification Loss')
ax[1].legend()

ax[2].plot(hist.history['regress_loss'], color='teal', label='regress loss')
ax[2].plot(hist.history['val_regress_loss'], color='orange', label='val regress loss')
ax[2].title.set_text('Regression Loss')
ax[2].legend()

plt.show()

# 11. Make Predictions

### 11.1 Make Predictions on Test Set

In [ ]:
test_data = test.as_numpy_iterator()

In [ ]:
test_sample = test_data.next()

yhat = facetracker.predict(test_sample[0])

fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = test_sample[0][idx]
    sample_coords = yhat[1][idx]
    
    if yhat[0][idx][0] > 0.9:
        cv2.rectangle(sample_image, 
                      tuple(np.multiply(sample_coords[0:2], [120,120]).astype(int)),
                      tuple(np.multiply(sample_coords[2:4], [120,120]).astype(int)), (255,0,0), 2)
    if yhat[0][idx][1] > 0.9:
        cv2.rectangle(sample_image, 
                      tuple(np.multiply(sample_coords[4:6], [120,120]).astype(int)),
                      tuple(np.multiply(sample_coords[6:8], [120,120]).astype(int)), (0,255,0), 2)
    
    ax[idx].imshow(sample_image)
    ax[idx].set_title(', '.join([f'{x:,.0%}' for x in yhat[0][idx]]))

### 11.2 Save the Model

In [ ]:
from tensorflow.keras.models import load_model
import cv2
import tensorflow as tf
import numpy as np

In [ ]:
# facetracker.save('facetracker.h5')

In [ ]:
facetracker = load_model('facetracker.h5')

#### 11.2.1 Single Photo

In [ ]:
# cap = cv2.VideoCapture(0)
# _, frame_captured = cap.read()

In [ ]:
final_test_wd = data_wd / 'final_test_images'
img_gen = final_test_wd.glob('*.jpg')

In [ ]:
frame_captured = cv2.imread(str(next(img_gen)))
plt.imshow(frame_captured)

frame = frame_captured[:, 80:640-80,:]

rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
resized = tf.image.resize(rgb, (120,120))

yhat = facetracker.predict(np.expand_dims(resized/255,0))
labels, sample_coords = [x[0] for x in yhat]
labels, sample_coords

if yhat[0][0][0] > 0.4:
    # Controls the main rectangle
    cv2.rectangle(frame, 
                  tuple(np.multiply(sample_coords[0:2], [450,450]).astype(int)),
                  tuple(np.multiply(sample_coords[2:4], [450,450]).astype(int)), 
                        (255,0,0), 2)
    # Controls the label rectangle
    cv2.rectangle(frame, 
                  tuple(np.add(np.multiply(sample_coords[0:2], [450,450]).astype(int), 
                                [0,-30])),
                  tuple(np.add(np.multiply(sample_coords[0:2], [450,450]).astype(int),
                                [80,0])), 
                        (255,0,0), -1)
    # Controls the text rendered
    cv2.putText(frame, 'brendan', tuple(np.add(np.multiply(sample_coords[0:2], [450,450]).astype(int),
                                           [0,-5])),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
if yhat[0][0][1] > 0.4:
    # Controls the main rectangle
    cv2.rectangle(frame, 
                  tuple(np.multiply(sample_coords[4:6], [450,450]).astype(int)),
                  tuple(np.multiply(sample_coords[6:8], [450,450]).astype(int)), 
                        (255,0,0), 2)
    # Controls the label rectangle
    cv2.rectangle(frame, 
                  tuple(np.add(np.multiply(sample_coords[4:6], [450,450]).astype(int), 
                                [0,-30])),
                  tuple(np.add(np.multiply(sample_coords[4:6], [450,450]).astype(int),
                                [80,0])), 
                        (255,0,0), -1)
    
    # Controls the text rendered
    cv2.putText(frame, 'kara', tuple(np.add(np.multiply(sample_coords[4:6], [450,450]).astype(int),
                                           [0,-5])),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

im = plt.imshow(frame)
im.axes.set_title(', '.join([f'{x:,.0%}' for x in labels]))
plt.show()

### 11.3 Real Time Detection

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _ , frame = cap.read()
    frame = frame[50:500, 50:500,:]
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120,120))
    
    yhat = facetracker.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    
    if yhat[0][0][0] > 0.9:
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[0:2], [450,450]).astype(int)),
                      tuple(np.multiply(sample_coords[2:4], [450,450]).astype(int)), 
                            (255,0,0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(sample_coords[0:2], [450,450]).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(sample_coords[0:2], [450,450]).astype(int),
                                    [80,0])), 
                            (255,0,0), -1)
        # Controls the text rendered
        cv2.putText(frame, 'brendan', tuple(np.add(np.multiply(sample_coords[0:2], [450,450]).astype(int),
                                               [0,-5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    if yhat[0][0][0] > 0.9:
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[4:6], [450,450]).astype(int)),
                      tuple(np.multiply(sample_coords[6:8], [450,450]).astype(int)), 
                            (255,0,0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(sample_coords[4:6], [450,450]).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(sample_coords[4:6], [450,450]).astype(int),
                                    [80,0])), 
                            (255,0,0), -1)
        
        # Controls the text rendered
        cv2.putText(frame, 'kara', tuple(np.add(np.multiply(sample_coords[4:6], [450,450]).astype(int),
                                               [0,-5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.imshow('EyeTrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()